Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

In [ ]:
dg = DataGroup('./cage.json', load=True)
run = 143
# cycle = 1197
str_query = f'run=={run} and skip==False'
# str_query = f'cycle=={cycle} and skip==False'
dg.fileDB.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.fileDB)

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.fileDB['runtype'])
runtype = runtype_list[0]

rt_min = dg.fileDB['runtime'].sum()
u_start = dg.fileDB.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['source'])[0]
    print(f'Radius: {radius}; Source Angle: {angle}')
# else:

In [ ]:
# Working with dsp or hit files?

# hit = True 
hit = False #ie working with dsp files

In [ ]:
# Working with calibrated or uncalibrated data?

# cal = True #calibrated data
cal = False #uncalibrated data

In [ ]:
# Working in your own directory or with the CAGE_lh5 directory?

user = False # CAGE_lh5 directory
# user = True # hit filesin my personal directory

In [ ]:
# Get relevant files

lh5_dir = dg.lh5_user_dir if user else dg.lh5_dir
# lh5_dir = '/global/homes/g/gothman/projecta/CAGE_lh5_joule'
print(lh5_dir)

# if hit files
if hit == True:
    file_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
    
else:
    file_list = lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']
    
print(f'lh5_dir: {lh5_dir}')
print(f'file list: {file_list}')

In [ ]:
# Create a dataframe

if (cal == True) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == True) and (hit == False):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')

elif (cal == False) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == False) and (hit == False):
    #df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')


else:
    print('dont know what to do here! need to specify if working with calibrated/uncalibrated data, or dsp/hit files')

# df_hit = lh5.load_dfs(file_list, ['trapEmax', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
# print(df_hit.columns)

# dsc = df[['bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)
print(df.columns)



In [ ]:
# lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
# print(lh5_dir)
# hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
# print(hit_list)
# df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEmax_cal', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# # df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# # print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [ ]:
# get raw files for wfs

raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']

print(raw_list)

# don't actually need this, just file list
# raw_df = lh5.load_dfs(raw_list, ['energy'], 'ORSIS3302DecoderForEnergy/raw')



In [ ]:
# select energy typ
# etype = 'trapEmax'
# etype = 'energy'
etype = 'trapEftp'

In [ ]:
# use baseline cut
bl_cut_lo, bl_cut_hi = 9700,9760

df_cut = df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()


#create dcr/E 
# df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

#create 10-90
df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

#create 50-90
df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

In [ ]:
#creat new DCR
# const = 0.0555
# df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

In [ ]:
# baseline cut
bl = df['bl']
bl_cut = (bl > 9700) & (bl < 9760)

# print(cut)
print(bl_cut.value_counts())

In [ ]:
tp_0 = df['tp_0']
print(tp_0)

In [ ]:
##### energy cut 
energy = df[etype]

if cal == True:
    # energy = df['trapEmax_cal']
    energy_cut = (energy > 1450) & (energy < 1470) # 1460 peak
    
else:
    # energy = df_hit['trapEmax']
    # energy_cut = (energy > 2805) & (energy < 2835) # 1460 peak, uncalibratedm trapEmax
    # energy_cut = (energy > 2000000) & (energy < 2500000) #where alphas seem to be in run 117, raw energy parameter
    energy_cut = (energy > 200) & (energy < 1000) #where low e weirdness is in DCR, in trapEftp
    
# print(type(energy))
# print(energy_cut)
print(energy_cut.value_counts())

In [ ]:
cut = bl_cut & energy_cut

In [ ]:
# for debugging

# print(cut)
print(cut.value_counts())



DCR Plots



In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'dcr_trap', 'dcr_trap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "DCR: {dcr:0.2f}"],
#                           legend_opts={'loc':"upper left"},
                          lines=['dcr'], # add hlines and vlines
                          x_lim=(38000, 80000) # x axis range
                         )

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]


In [ ]:
browser.draw_next()
# plt.ylim(0, 100)
# plt.plot(4090)

Energy Plots

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_trap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected',  'trapEftp = {trapEftp:0.2f}'],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEftp'], # add hlines and vlines
                          x_lim=(0, 80000) # x axis range
                         )

In [ ]:
browser2.draw_next()
# plt.ylim(4000, 4500)

TP_0

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser3 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_atrap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected',  't0 = {tp_0:0.2f}'],
#                           legend_opts={'loc':"upper left"},
                          lines=['tp_0'], # add hlines and vlines
                          x_lim=(0, 50000) # x axis range
                         )

In [ ]:
browser3.draw_next()
plt.ylim(-10, 100) 

WF_psd

LF PD

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser4 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_psd', 'wf_psd'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']}],
                          legend=['lf_max = {lf_max:0.2f}', 'wf_psd'], #{tp_0:0.2f}
#                           legend_opts={'loc':"upper left"},
                          lines=['lf_max'], # add hlines and vlines
                          x_unit = 'khz',
                          x_lim=(0, 200), # x axis range,
                          verbosity=2
                         )

In [ ]:
browser4.draw_next()
# plt.ylim(0, 2000)

HF PSD

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser5 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_psd', 'wf_psd'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']}],
                          legend=['hf_max = {hf_max:0.2f}', 'wf_psd'], #{tp_0:0.2f}
#                           legend_opts={'loc':"upper left"},
                          lines=['hf_max'], # add hlines and vlines
                          x_unit = 'mhz',
                          x_lim=(0, 50), # x axis range,
                          verbosity=2
                         )

In [ ]:
browser5.draw_next()
# plt.ylim(0, 2000)

Baseline Slope

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser6 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'bl_trap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "bl_slope: {bl_slope:0.2f}"],
#                           legend_opts={'loc':"upper left"},
                          lines=['bl_slope'], # add hlines and vlines
                          x_lim=(0, 80000) # x axis range
                         )

In [ ]:
browser6.draw_next()
# plt.ylim(0, 20000)